<a href="https://colab.research.google.com/github/jesusGS01/TGINE/blob/main/3_3_scrapyUM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sesión 3.3 - Scrapy - Web noticias UM
En este ejemplo vamos a definir un Spider sencillo para obtener las noticias de la Universidad de Murcia que se encuentran en la URL https://www.um.es/web/sala-prensa/notas-de-prensa/historico?type=com.liferay.journal.model.JournalArticle&category=4753007

Para eso deberemos ver el código de la página y su estructura para encontrar la información que queremos extraer.

Una vez sabemos eso, podemos definir nuestra clase UMSpider.


In [1]:
!pip3 install -U scrapy

In [2]:
import scrapy
import sys
import json
import locale
import time
import random
from bs4 import BeautifulSoup

# Para cada crawler que nos definimos nos debemos crear una clase Spider que debe heredar de la clase scrapy.Spider

class UMSpider (scrapy.Spider):
    name = 'unimurcia'

    # Decimos que el dominio válido es el de la UM
    allowed_domains = ['www.um.es']

    # podemos definir las páginas de inicio
    start_urls = ['https://www.um.es/web/sala-prensa/notas-de-prensa/historico?type=com.liferay.journal.model.JournalArticle&category=4753007']

    # para evitar que el sitio te bloquee por usar scrapy es interesante cambiar el USER_AGENT
    # El user agent por defecto de Scrapy cuando hace una petición es
    # Scrapy/VERSION (+https://scrapy.org)
    custom_settings = {
        'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    }

    # debemos de implementar este método que se llamará para cada una de las páginas que se vayan analizando
    def parse (self, response):
        """
        @inherit

        @param self
        @param response
        """

        # Guardamos la URL del sitio que se está visitando
        url = str(response.request.url).strip()

        # Cogemos el contenido relevante y para eso debemos usar selectores CSS
        for article in response.css ('.NotaDePrensa'):
            # Cogemos el contenido del título
            title = str (article.css ('h1::text').get ()).strip()
            title = BeautifulSoup (title, 'html.parser').get_text().strip()
            # Cogemos el contenido del contenido
            content = "".join (article.css ('.mt-5').get ())
            content = BeautifulSoup (content, 'html.parser').get_text().strip().replace("\"","")

            # Cogemos también la fecha si existe
            date = str(article.css ('#NP-fecha').get().strip())
            if (date):
                date = BeautifulSoup (date, 'html.parser').get_text().strip()

            print ("-------------------------")
            print (url)
            print (title)
            print (content)
            print (date)
            print ("-------------------------")

            data = {
                'url' : url,
                'title': title,
                'content': content,
                'date': date
            }

            filename = str(random.random()).replace(".","") + ".json"

            # Guardamos el documento si tiene contenido y título
            if content and title:
                with open ('umu/' + filename, 'w') as f:
                    json.dump (data, f, indent = 4)


        # Obtenemos todas las otros links de la página representados por la etiqueta <a>
        url_in_current_document = response.css ('a')
        for next_page in url_in_current_document:
            # Para limitar que solamente se parseen las noticias dentro de 'https://www.um.es/web/sala-prensa/'
            url_str = str(next_page.css('::attr(href)').get())
            if "www.um.es/web/sala-prensa/" in url_str:
              # obtenemos el atributo href de la etiqueta <a> y parseamos la página
              yield response.follow (next_page, self.parse)


## Apartado 1.2
Para poder lanzar el Spider necesitamos que ejecutar el siguiente código donde se configuará y lanzará el proceso.
Hay que hacer notar que solamente se puede lanzar un proceso por cada sesión en Jupyter notebook es por eso por lo que se recomienda exportar el código en un script de Python .py para poder ejecutarlo desde la línea de comandos.

In [3]:
import scrapy, os
from scrapy.crawler import CrawlerProcess

# Creamos un proceso de Crawler podemos poner distintas settings que están definidas en la documentación.
# Entre ellas podemos ocular los logs del proceso de Crawling.
process = CrawlerProcess(settings={
    "LOG_ENABLED": False,
    # Used for pipeline 1
})

# Como se ha definido anteriormente en el RSSCrawler, los ficheros se van a almacenar en la carpeta "rss"
# Comprobamos que existe la carpeta y si no existe la creamos
if (os.path.exists('umu')== False):
    os.mkdir('umu')

# Creamos el proceso con el RSSSpider
process.crawl(UMSpider)
# Ejecutamos el Crawler
process.start()

INFO:scrapy.utils.log:Scrapy 2.11.0 started (bot: scrapybot)
INFO:scrapy.utils.log:Versions: lxml 4.9.3.0, libxml2 2.10.3, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.2, Twisted 22.10.0, Python 3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0], pyOpenSSL 23.2.0 (OpenSSL 3.1.4 24 Oct 2023), cryptography 41.0.5, Platform Linux-5.15.120+-x86_64-with-glibc2.35
INFO:scrapy.addons:Enabled addons:
[]
/usr/local/lib/python3.10/dist-packages/scrapy/utils/request.py:254: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.

See the documentation of the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting for information on how to handle this deprecation.
  return cls(crawler)
DEBUG:scrap

-------------------------
https://www.um.es/web/sala-prensa/-/la-facultad-de-comunicaci%C3%B3n-y-documentaci%C3%B3n-expone-el-proyecto-fotogr%C3%A1fico-vulnerabilidad-queer-/1.0?redirect=%2Fweb%2Fsala-prensa%2Fnotas-de-prensa%2Fhistorico%3Ftype%3Dcom.liferay.journal.model.JournalArticle%26category%3D4753007
La Facultad de Comunicación y Documentación expone el proyecto fotográfico 'Vulnerabilidad queer'
La Facultad de Comunicación y Documentación de la Universidad de Murcia ha inaugurado la exposición 'Vulnerabilidad Queer', un proyecto realizado por el estudiante de 4º de Comunicación Audiovisual José Manuel Pérez Molina. 
La muestra consiste en una mirada profunda a las vidas de la comunidad LGTBI+ a través de 19 fotografías, extraídas del foto-libro homónimo y que pretende transmitir la resistencia que reside en las personas queer.

Este proyecto fue realizado como parte de la asignatura 'Análisis de los discursos audiovisuales y de sus efectos sociales', impartida por la profesora 

DEBUG:scrapy.core.engine:Crawled (200) <GET https://www.um.es/web/sala-prensa/-/el-rector-de-la-umu-valora-de-forma-positiva-que-el-futuro-plan-de-financiaci%C3%B3n-incluya-fondos-por-la-consecuci%C3%B3n-de-objetivos-estrat%C3%A9gicos-para-la-regi%C3%B3n/1.4?redirect=%2Fweb%2Fsala-prensa%2Fnotas-de-prensa%2Fhistorico%3Ftype%3Dcom.liferay.journal.model.JournalArticle%26category%3D4753007> (referer: https://www.um.es/web/sala-prensa/notas-de-prensa/historico?type=com.liferay.journal.model.JournalArticle&category=4753007)
DEBUG:scrapy.core.engine:Crawled (200) <GET https://www.um.es/web/sala-prensa/-/la-universidad-de-murcia-reflexiona-en-un-congreso-sobre-la-cultura-de-la-paz/1.0?redirect=%2Fweb%2Fsala-prensa%2Fnotas-de-prensa%2Fhistorico%3Ftype%3Dcom.liferay.journal.model.JournalArticle%26category%3D4753007> (referer: https://www.um.es/web/sala-prensa/notas-de-prensa/historico?type=com.liferay.journal.model.JournalArticle&category=4753007)
DEBUG:scrapy.core.engine:Crawled (200) <GET htt

-------------------------
https://www.um.es/web/sala-prensa/-/la-universidad-de-murcia-inaugura-la-vii-edici%C3%B3n-del-programa-de-inclusi%C3%B3n-todos-somos-campus-/1.0?redirect=%2Fweb%2Fsala-prensa%2Fnotas-de-prensa%2Fhistorico%3Ftype%3Dcom.liferay.journal.model.JournalArticle%26category%3D4753007
La Universidad de Murcia inaugura la VII edición del programa de inclusión ‘Todos Somos Campus’
La Universidad de Murcia ha inaugurado este martes la VII edición del programa ‘Todos Somos Campus’, destinado a formación para el empleo para jóvenes con discapacidad intelectual. El rector, José Luján; la vicerrectora de Estudiantes y Empleo y el consejero Medio Ambiente, Universidades, Investigación y Mar Menor, Juan María Vázquez, han sido los encargados de dar la bienvenida a este nuevo curso al alumnado, que a lo largo de los próximos meses recibirán formación en las instalaciones de la Universidad de Murcia y tendrán contactos con el mercado laboral.
En esta edición participan 18 estudian

DEBUG:scrapy.core.engine:Crawled (200) <GET https://www.um.es/web/sala-prensa/-/vuelve-la-segunda-edici%C3%B3n-del-festival-umurgentes-como-trampol%C3%ADn-para-el-talento-joven-murciano> (referer: https://www.um.es/web/sala-prensa/actualidad)
DEBUG:scrapy.core.engine:Crawled (200) <GET https://www.um.es/web/sala-prensa/-/la-universidad-de-murcia-corre-este-jueves-sin-resistencias-> (referer: https://www.um.es/web/sala-prensa/actualidad)
DEBUG:scrapy.core.engine:Crawled (200) <GET https://www.um.es/web/sala-prensa/-/la-c%C3%A1tedra-del-sue%C3%B1o-gomarco-de-la-umu-convoca-su-primeros-premios-de-investigaci%C3%B3n-y-divulgaci%C3%B3n> (referer: https://www.um.es/web/sala-prensa/actualidad)
DEBUG:scrapy.core.engine:Crawled (200) <GET https://www.um.es/web/sala-prensa/-/la-sede-electr%C3%B3nica-de-la-universidad-de-murcia-finalista-en-los-premios-cnis-2023> (referer: https://www.um.es/web/sala-prensa/actualidad)


...

INFO:scrapy.spidermiddlewares.httperror:Ignoring response <404 https://www.um.es/web/sala-prensa/-/la-universidad-de-murcia-crea-la-sede-permanente-de-extensi%C3%B3n-universitaria-en-la-uni%C3%B3n>: HTTP status code is not handled or not allowed
DEBUG:scrapy.core.engine:Crawled (200) <GET https://www.um.es/web/sala-prensa/-/conferencia-sobre-la-escuela-cajal-en-el-marco-de-las-jornadas-una-educaci%C3%B3n-para-el-siglo-xxi-coorganizadas-por-la-umu-y-aferem> (referer: https://www.um.es/web/sala-prensa/notas-de-prensa?p_p_id=com_liferay_asset_publisher_web_portlet_AssetPublisherPortlet_INSTANCE_GNs6DmEJkR1y&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&_com_liferay_asset_publisher_web_portlet_AssetPublisherPortlet_INSTANCE_GNs6DmEJkR1y_delta=20&p_r_p_resetCur=false&_com_liferay_asset_publisher_web_portlet_AssetPublisherPortlet_INSTANCE_GNs6DmEJkR1y_cur=41)
DEBUG:scrapy.core.engine:Crawled (200) <GET https://www.um.es/web/sala-prensa/-/fallece-a-los-101-a%C3%B1os-margarita-zielinski-picqu

-------------------------
https://www.um.es/web/sala-prensa/-/la-umu-publica-un-inventario-ilustrado-de-los-fondos-del-museo-de-biolog%C3%ADa-jos%C3%A9-loustau-
La UMU publica un inventario ilustrado de los fondos del museo de Biología ‘José Loustau’
El pasado 31 de enero, en el acto de apertura de la XXXV Semana de Biología, se presentó el libro ‘Museo José Loustau. Inventario 2019’, un Catálogo-Inventario ilustrado de los fondos del museo José Loustau, ubicado en la planta baja de la Facultad de Biología de la Universidad de Murcia (UMU), en el campus de Espinardo.
“Este Libro-inventario deja constancia de un legado patrimonial que posteriores generaciones de científicos murcianos deberían conservar y, por supuesto, describir con nuevas tecnologías: digitalización 3D, libro electrónico, etc.”, expone Manuel Acosta, catedrático emérito de la Facultad de Biología de la Universidad de Murcia (UMU). “A pesar de las vicisitudes vividas, la guerra civil o los traslados, hoy se conserva un 

DEBUG:scrapy.core.engine:Crawled (200) <GET https://www.um.es/web/sala-prensa/notas-de-prensa?p_p_id=com_liferay_asset_publisher_web_portlet_AssetPublisherPortlet_INSTANCE_GNs6DmEJkR1y&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&_com_liferay_asset_publisher_web_portlet_AssetPublisherPortlet_INSTANCE_GNs6DmEJkR1y_delta=20&p_r_p_resetCur=false&_com_liferay_asset_publisher_web_portlet_AssetPublisherPortlet_INSTANCE_GNs6DmEJkR1y_cur=198> (referer: https://www.um.es/web/sala-prensa/notas-de-prensa?p_p_id=com_liferay_asset_publisher_web_portlet_AssetPublisherPortlet_INSTANCE_GNs6DmEJkR1y&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&_com_liferay_asset_publisher_web_portlet_AssetPublisherPortlet_INSTANCE_GNs6DmEJkR1y_delta=20&p_r_p_resetCur=false&_com_liferay_asset_publisher_web_portlet_AssetPublisherPortlet_INSTANCE_GNs6DmEJkR1y_cur=223)


-------------------------
https://www.um.es/web/sala-prensa/-/un-proyecto-de-la-umu-implementa-una-alternativa-natural-a-los-conservantes-qu%C3%ADmicos-en-alimentos
Un proyecto de la UMU implementa una alternativa natural a los conservantes químicos en alimentos
El alto consumo de productos cárnicos requiere una explotación de la industria ganadera, lo que supone un estrés para los animales que repercute en el producto y el consumidor. La ganadería intensiva tiene, además, un alto impacto ambiental. Para paliar dicha situación se están desarrollando diferentes modelos alimenticios en los que incorporar nuevas fuentes de proteínas que eviten la explotación animal.
En el seno de la Facultad de Veterinaria de la Universidad de Murcia (UMU), se ha llevado a cabo, durante los últimos años, el estudio de diversos ingredientes característicos de la dieta mediterránea para que actúen como sustitutos de conservantes sintéticos en la elaboración de productos cárnicos manufacturados.
“Las tendenc

INFO:scrapy.statscollectors:Dumping Scrapy stats:
{'downloader/request_bytes': 209992,
 'downloader/request_count': 220,
 'downloader/request_method_count/GET': 220,
 'downloader/response_bytes': 20254182,
 'downloader/response_count': 220,
 'downloader/response_status_count/200': 217,
 'downloader/response_status_count/404': 3,
 'dupefilter/filtered': 4095,
 'elapsed_time_seconds': 15.971466,
 'finish_reason': 'shutdown',
 'finish_time': datetime.datetime(2023, 10, 26, 17, 30, 12, 61784, tzinfo=datetime.timezone.utc),
 'httpcompression/response_bytes': 134034,
 'httpcompression/response_count': 3,
 'httperror/response_ignored_count': 3,
 'httperror/response_ignored_status_count/404': 3,
 'log_count/DEBUG': 224,
 'log_count/INFO': 14,
 'memusage/max': 170029056,
 'memusage/startup': 170029056,
 'request_depth_max': 6,
 'response_received_count': 220,
 'scheduler/dequeued': 220,
 'scheduler/dequeued/memory': 220,
 'scheduler/enqueued': 913,
 'scheduler/enqueued/memory': 913,
 'start_tim